# Warmup for tensorflow

## Fibonacci numbers

In [68]:
import tensorflow as tf
import collections


def fibonacci_number(n_):
    """ calculates the fibonacci numbers with tf
         j = 1
         k = 0
         n = 10 # the searched fibonacci number
         for i in range(1, n):
             j = j + k
             k = j
         print('fibonacci number for ' + n + ' = ' j)
    """
    Pair = collections.namedtuple('Pair', 'j, k')
    ijk_0 = (tf.constant(1), Pair(tf.constant(1), tf.constant(0)))
    n = tf.placeholder(tf.int32)
    c = lambda i, p: i < n
    b = lambda i, p: (i + 1, Pair(p.j+ p.k, p.j))
    ijk_final = tf.while_loop(c, b, ijk_0)

    with tf.Session() as sess:
        result = sess.run(ijk_final, {n: n_})
        return result[1][0]


fib_n = []
for i in range(1, 15):
    fib_n.append(fibonacci_number(i))

print(fib_n)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377]
